# EEGNet

### Import Library

In [1]:
import torch 
import itertools
import pandas as pd 
import pickle 
import numpy as np
import mne

torch.manual_seed(0)

### Select Dataset 


In [3]:
datasets = ['aBNCI2014001R.pickle', 'aBNCI2014004R.pickle', 'aWeibo2014R.pickle', 'aPhysionetRR.pickle']

In [4]:
def load_data(filename):
    with open(filename, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [5]:
data = load_data(datasets[0])

In [7]:
class_name = ['left_hand', 'right_hand']

## GPU Device

In [8]:
import torch
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
    print('Your GPU device name :', torch.cuda.get_device_name())    

Your GPU device name : NVIDIA GeForce RTX 3060 Laptop GPU


### EEGNet Model

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [10]:
class EEGNet(nn.Module):
    def __init__(self, n_channels, n_time_samples, n_classes):
        super(EEGNet, self).__init__()
        # Layer 1
        self.first_conv = nn.Sequential(
            nn.Conv2d(1, 16, (1, n_time_samples//2), stride=(1, 1), padding=(0, n_time_samples//4)),
            nn.BatchNorm2d(16, eps=1e-05, momentum=0.1)
        )
        # Layer 2
        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(16, 32, (n_channels, 1), stride=(1, 1), groups=16),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1),
            nn.ELU(alpha=1.0),
            nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0),
            nn.Dropout(p=0.5)
        )
        # Layer 3
        self.separable_conv = nn.Sequential(
            nn.Conv2d(32, 32, (1, 16), stride=(1, 1), padding=(0, 8)),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1),
            nn.ELU(alpha=1.0),
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0),
            nn.Dropout(p=0.5)
        )
        
        self.classify = nn.Sequential(
            nn.Linear(in_features=32, out_features=n_classes, bias=True)
        )

    def forward(self, x):
        x = self.first_conv(x)
        x = self.depthwise_conv(x)
        x = self.separable_conv(x)
        x = torch.mean(x, 3)
        x = x.permute(0, 2, 1).contiguous()
        x = self.classify(x)
        return x.squeeze()